<a href="https://colab.research.google.com/github/ahmedynah/Epsilon-AI/blob/main/Web_Scraping_Lecture_10_Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Task 1
---------
#####Scrap the USD To EGP Exchange rate from this website
#####https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html
#####and then use it to make a software that takes amount of USD Dollars from the user and calculate how much will it cost in EGP.

In [2]:
import requests
import csv
from bs4 import BeautifulSoup
import time

In [4]:
amount = float(input("amount in USD: "))
url = 'https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html'
res = requests.get(url)
soap = BeautifulSoup(res.text, 'html.parser')

amount in USD: 5


In [5]:
rate = float(soap.find('', attrs = {'id':"shd2b;"}).get_text())
print(f"{amount} USD = {amount * rate} EGP")

5.0 USD = 78.5005 EGP


Tassk 2
---------
#####Scrap the weather temperature, humidity, visibility, air pressure, wind speed, datetime data from this website
#####https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt
#####for 10 times one read in a minute (so basicly the process takes 10 minutes)and put them into a CSV file

In [ ]:
def get_weather(url):
  
  res = requests.get(url)
  
  soap = BeautifulSoup(res.text, 'html.parser')
  temp = soap.find('div', attrs={"class": "temp"}).get_text()
  stats = soap.find('div', attrs={"class": "stats"}).get_text().replace("Rel. ","")
  
  stats = stats.replace('Few Clouds at 760m',"")
  stats = stats.replace(" |","").splitlines()
  
  stats_headers, stats_values = [header.strip()for header in stats[0::2]], [value.strip() for value in stats[1::2]]
  
  stats_headers.insert(0,"temp")
  stats_values.insert(0,temp)

  return stats_headers, stats_values

In [ ]:
def repeate_every_x_min_for_10_times(x):
    with open("weather.csv", "w") as f:
      url = 'https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt'
      headers, values = get_weather(url)
      writer = csv.writer(f)
      writer.writerow(headers)
      for _ in range(10):
         url = 'https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt'
         headers, values = get_weather(url)
         writer.writerow(values)
    time.sleep(x * 60)

In [ ]:
repeate_every_x_min_for_10_times(1)

#Task 3
---------
#####Scrap the books (name, price, rate) for each category and put them into a CSV file
#####https://books.toscrape.com/

In [6]:
url = 'https://books.toscrape.com/catalogue/category/books_1/index.html'

In [7]:
res = requests.get(url)

In [99]:
soap = BeautifulSoup(res.text, 'html.parser')
index = soap.find('a', attrs={'href':'index.html'})
cat_trav = index.find_next_sibling('ul')
categories = cat_trav.get_text()
categories = categories.split()
links_of_each_cat = cat_trav.findAll('li')
links_of_each_cat = [link.find('a').get('href').replace("..",'https://books.toscrape.com/catalogue/category')
 for link in links_of_each_cat]


In [100]:
def get_books_info(link):
  res = requests.get(link)
  soap = BeautifulSoup(res.text,"html.parser")
  books = soap.findAll('article', attrs={'class':'product_pod'})
  books_info = [
     [book.find('h3').find('a').get('title')
     ,book.find('p').get('class')[1] + " stars"
     ,book.find('p',attrs={'class':'price_color'}).get_text().replace("Â","")
     ,book.find('h3').find('a').get('href').replace("../../..","https://books.toscrape.com/catalogue")]
  for book in books
  ]
  return books_info

In [118]:
def write_in_csv():
  with open("books.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['Name', 'Rate', "Price", "Link", "Category"])
    i = 0
    for cat_link in links_of_each_cat:
      books = traverse_pages(cat_link)
      for book in books:
        book.append(categories[i])
        writer.writerow(book)
      i += 1

In [119]:
def are_there_multiple_pages(link):
  res = requests.get(link)
  soap = BeautifulSoup(res.text,'html.parser')
  multiple_pages = soap.find('ul',attrs={'class':'pager'})
  if multiple_pages != None:
    number_of_pages = multiple_pages.findAll('li')[0].get_text()
    number_of_pages = number_of_pages.strip().split()[-1]
    return True, int(number_of_pages), multiple_pages
  else:
    return False, 0, 0

In [120]:
def get_next_page(link, pager):
  next_page = pager.find('li', attrs={'class':'next'}).find('a').get('href')
  link = link.replace("index.html",next_page)
  return link

In [121]:
def traverse_pages(link):
  
  all_books = []
  check, no_pages, pager = are_there_multiple_pages(link)
  if check:    
    for _ in range(no_pages):
      all_books.extend(get_books_info(link))
      link = get_next_page(link,pager)
      res = requests.get(link)
      # print(link)
  else:
    all_books.extend(get_books_info(link))
  # print(all_books)
  return all_books

In [ ]:
traverse_pages(links_of_each_cat[1])

In [123]:
write_in_csv()